In [1]:
"""Write separate decorators that:
1. allows a function to run only on odd seconds - 100pts
2. log - 100pts
3. authenticate - 300pts
4. timed (n times) - 100pts
5. Provides privilege access (has 4 parameters, based on privileges (high, mid, low, no), gives access to all 4, 3, 2 or 1 params) - 200pts
Write our htmlize code using inbuild singledispatch - 100pts"""

'Write separate decorators that:\n1. allows a function to run only on odd seconds - 100pts\n2. log - 100pts\n3. authenticate - 300pts\n4. timed (n times) - 100pts\n5. Provides privilege access (has 4 parameters, based on privileges (high, mid, low, no), gives access to all 4, 3, 2 or 1 params) - 200pts\nWrite our htmlize code using inbuild singledispatch - 100pts'

In [73]:
import time
from time import perf_counter
import pandas as pd
from functools import wraps
from datetime import datetime
from numbers import Integral
from collections.abc import Sequence, Set 
from decimal import Decimal
from functools import singledispatch
from html import escape
import time


def run_oddsec(fn):

    def check_run(*args, sec = None, **kwargs):
        if sec is None:
            sec = datetime.now().second
        print('time checknow', sec)
        if sec % 2 != 0:
            print("Func runs now - odd sec")
            return fn(*args, **kwargs)
        else:
            print('Even second, not running now')
    return check_run

@run_oddsec
def add(*args):
    return sum(args)

# HTMLIZE

@singledispatch
def htmlize(obj):
    return escape(str(obj))

@htmlize.register(Integral)
def html_real(a):
    return f'{round(a, 2)}'

@htmlize.register(float)
def html_real(a):
    return f'{round(a, 2)}'

@htmlize.register(Decimal)
def html_real(a):
    return f'{round(a, 2)}'

@htmlize.register(str)
def html_str(s):
    return escape(s).replace('\n', '<br/>\n')

@htmlize.register(tuple)
@htmlize.register(list)
def html_sequence(l):
    items = (f'<li>{htmlize(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(dict)
def html_dict(d):
    items = (f'<li>{k}={v}</li>' for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

# Log of functions

def logger(fn):
    '''Decorator takes care of function stats'''

    stats_dict = dict()

    @wraps(fn)
    def store_stats(*args, **kwargs):
        func_started = time.perf_counter()
        stats_dict["func_started"] = datetime.now()
        fn(*args, **kwargs)
        stats_dict["func_ended"] = datetime.now()
        func_ended = time.perf_counter()
        stats_dict["run_time"] = func_ended - func_started
        stats_dict["func_name"] = fn.__name__
        stats_dict["func_docstr"] = fn.__doc__

        return stats_dict

    return store_stats

@logger
def check_logger():
    a = 1
    for i in range(1000):
        a = a**i
        

# Authenticate
def set_password(password=None):
    '''Sets the default password if no values are supplied
    :args password: str
    :returns inner: closure function
    '''
    def inner():
        nonlocal password
        if password == None:
            password = 'tsaiRocks123'
        return hash(password)
    return inner

@authenticate
def add_auth(*args):
    return sum(args)

def authenticate(fn):
    '''Decorator to check for the authentication before accessing any functions'''

    def check_creds(user_password, in_password, *args, **kwargs):
        if user_password() != hash(in_password):
            print('Bhai isko hack karke kya hi achive kar lega life mai')

        else:
            print('suer Authenticated')
            print(f"Function {fn.__name__} is called")
            return fn(*args)
    return check_creds



# Admin stuff

def sod(fn):
    '''Maps the user's and returns information applicable to him'''
    dict_priv = {

    1: ["Emp_Names"],
    2: ["Emp_Names", "Priviledge", "Money"],
    3: ["Emp_Names", "Priviledge", "Money", "ID"],
    4: ["Emp_Names", "Priviledge", "ID", "Useless_Info"]
    }
    
    @wraps(fn)
    def check_priv(emp_name, df):
        '''Returns dataframe output based on user's priviledges'''
        if emp_name in df.values:
            priv_no = df.loc[df['Emp_Names'] == emp_name]["Priviledge"]
            priv_no = (priv_no.item())
            return df[dict_priv[priv_no]]
        else:
            print('Employee data not in system, please double check')
    return check_priv

@sod
def check_userdata(emp_name):
    return None


def timed(n_times: int):
    '''Logs the function for n times'''
    def outer(fn):
        
        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(n_times):
                timer_start  = perf_counter()
                fn(*args, **kwargs)
                total_elapsed  += (perf_counter() - timer_start)
            
            avg_time = total_elapsed/n_times
            print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_time, n_times))
            return avg_time
        return inner
    return outer

@timed(100)
def square(a):
    '''Tests timed funtion by squaring a no n times'''
    a = a**2

In [2]:
from time import perf_counter
def timed(n_times: int):
    '''Logs the function for n times'''
    def outer(fn):
        

        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(n_times):
                timer_start  = perf_counter()
                fn(*args, **kwargs)
                total_elapsed  += (perf_counter() - timer_start)
            
            avg_time = total_elapsed/n_times
            print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_time, n_times))
            return avg_time
        return inner
    return outer

@timed(100)
def square(a):
    '''Tests timed funtion by squaring a no n times'''
    a = a**2

In [22]:
@timed(100)
def square(a):
    a = a**2

In [3]:
def check_timed():
    total = 0
    assert type(square(2)) == float, "Checks the return value of avg time taken for 100 reps"
check_timed()

Avg Run time: 0.000001s (100 reps)


In [25]:
type(a)

float

In [59]:
def test_add_evensec():
    assert add(1,3, sec=34) == None, "Function runs at even second, not cool"
    assert add(1,3, sec=33) == 4, "Function runs at odd second, so cool"
    assert add(1,2,3) == 6 or add(1,2,3) == None, "Function works perfectly fine"

def test_logger():
    '''Checks if logger function returns a dictionary'''
    assert type(check_logger()) == dict, "No logging dictionary returned"


def test_sod():
    '''Creates a dataframe that follows the template and adds data to check sod function'''

    data = { "Emp_Names":['Nikhil','Naman', 'Ad', 'SupAd'],
            "ID": [1,2,3,4],
            "Priviledge": [1,2,3,4], 
            "Money": ["Bohotkam", "Bohotzada", "Mat hi poocho", "Paisa kya hota hai"],
            "Useless_Info": ['sdfsadg', 'srtwtrew', 'kyukuiuy', 'gjkgjh']}

    df = pd.DataFrame(data, columns = ['Emp_Names', "Priviledge", "Money", "ID", "Useless_Info"])
    assert check_userdata('Nikhil', df).equals(df[['Emp_Names']]) == True
    assert check_userdata('Naman', df).equals(df[['Emp_Names',"Priviledge", "Money"]]) == True
    assert check_userdata('Ad', df).equals(df[['Emp_Names',"Priviledge", "Money", "ID"]]) == True
    assert check_userdata('SupAd', df).equals(df[['Emp_Names',"Priviledge", "ID",  "Useless_Info"]]) == True
    

def test_authentication():
    '''Checks the authentication by setting a password and calling the function'''
    user_password = set_password()
    assert add_auth(user_password, 'tsaiRocks123', 1,2) == 3
    assert add_auth(user_password, 'dfdssdf', 1,2) == None


def test_htmlize():
    '''Validates different html syntax implementations of htmlize functions'''
    assert htmlize([2,3]) == '<ul>\n<li>2</li>\n<li>3</li>\n</ul>'
    assert htmlize(1/3) == str(round(1/3,2))
    assert htmlize({1:1,2:2}) == '<ul>\n<li>1=1</li>\n<li>2=2</li>\n</ul>'


def check_timed():
    '''Checks the returned values of timed function'''
    assert type(square(2)) == float, "Checks the return value of avg time taken for 100 reps"


time checknow 34
Even second, not running now
time checknow 33
Func runs now - odd sec
time checknow 0
Even second, not running now
time checknow 0
Even second, not running now
suer Authenticated
Function add_auth is called
Bhai isko hack karke kya hi achive kar lega life mai


In [71]:
def test_htmlize():
    '''Validates different html syntax implementations of htmlize functions'''
    assert htmlize([2,3]) == '<ul>\n<li>2</li>\n<li>3</li>\n</ul>'
    assert htmlize(1/3) == str(round(1/3,2))
    assert htmlize({1:1,2:2}) == '<ul>\n<li>1=1</li>\n<li>2=2</li>\n</ul>'

test_htmlize()

In [69]:
htmlize({1:1,2:2})

'<ul>\n<li>1=1</li>\n<li>2=2</li>\n</ul>'